In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot
from tensorflow.keras import backend as K
from tensorflow.python.framework import ops
from tensorflow.keras.regularizers import l1,l2
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    LSTM,
    Dense,
    Input,
    Add
)

import math

In [ ]:
def mape(gt,pred):
    return np.abs((gt-pred)/gt)
    

In [ ]:
#----------change path according to local directories--------

window_sz = 28
horizon = 7
expert_path='../expert_preds/COVID/'

data=np.asarray(pd.read_csv('/data/COVID/covid_data.csv',header=None))


### Prepare data ---> Training and test data for LSTM and FourTheta

In [ ]:
# Prepare data for LSTM. Transorm time series into a supervised learning problem
def prepareData(ts): 
    
    samples = []
    labels = []
    minmaxtracker = []
    
    for i in range(len(ts)-window_sz-horizon+1):
        
        x = np.asarray(ts[i:i+window_sz])
        
        is_all_zero = np.all((x == 0))
     
        y = np.asarray(ts[i+window_sz: i+window_sz + horizon])
        
        if x.max() == x.min():
            x_np = 2*(((x - x.min()) / (x.max())) - 0.5)

            
        else:            
            x_np = 2*(((x - x.min()) / (x.max() - x.min())) - 0.5)


        samples.append(x_np)
        labels.append(y)
        
        minmaxtracker.append((i, x.min(), x.max()))
   
    return np.array(samples), np.array(labels)

In [ ]:
#Prepare test data
def prepareTestData(ts): 

    samples = []
    labels = []
    minmaxtracker = []

#     length=78 - window_sz
    length=50
    loop=length//horizon
    extra = length%horizon
    
    
    x=np.zeros([loop+1,window_sz])  
    y=np.zeros([loop+1,horizon])
    for i in range(loop):
        
        x[i,:] = ts[i*horizon:(i*horizon)+window_sz]
        y[i,:] = ts[(i*horizon)+window_sz:(i*horizon)+window_sz+horizon]
    x[loop,:] = ts[(loop)*horizon:((loop)*horizon)+window_sz]
    y[loop,:extra] = ts[((loop)*horizon)+window_sz:((loop)*horizon)+window_sz+horizon]
    min_in=np.min(x,axis=1).reshape(x.shape[0],1)
    max_in=np.max(x,axis=1).reshape(x.shape[0],1)
       

    denom=max_in-min_in
    a = np.where(denom == 0)[0]
    denom[a] = max_in[a] 
    a = np.where(denom == 0)[0]
    if a.size>0:
        denom[a]=-1
    x= 2*(((x - min_in) / (max_in - min_in)) - 0.5)
      
        
        
        
        
    
    return x, y

In [ ]:
# read csv file with inpatients column only

df = pd.read_csv('/home/chatta/Health_datas/Project_Time-Series-Forecasting-main/Covid/Data/COVID_confirmed_latest.csv', header=0)


In [ ]:
df = df[:110]
df.drop(['date'], axis=1, inplace=True)

In [ ]:

train_df = df[:60]
test_df = df[32:110]

In [ ]:
df.shape

In [ ]:
mape_ts = []
for index in range(25):
    
    print("index", index)    
    fileNumber =index + 2
    train_column = train_df.iloc[:, index]
    train_ts = train_column.values.astype(float)

    test_column = test_df.iloc[:, index]
    test_ts = test_column.values.astype(float)

    X_train, y_train = prepareData(train_ts)
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    y_train = y_train.reshape(y_train.shape[0], y_train.shape[1])
    
    X_test, y_test = prepareTestData(test_ts)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
    y_test = y_test.reshape(y_test.shape[0], y_test.shape[1])
    
    expert_train_df = pd.read_csv(expert_path+'train_'+str(fileNumber-1)+'.csv',index_col=[0], header=0)
    expert_train_df.dropna(inplace=True)  

    expert_train = np.asarray(expert_train_df).reshape(-1, horizon, 1)
    
    expert_test_df = pd.read_csv(expert_path+'test_'+str(fileNumber-1)+'.csv',index_col=[0], header=0)

    expert_test = np.asarray(expert_test_df).reshape(-1, horizon, 1)
    

       
    train_expert_combined = np.append(X_train, expert_train, axis=1)
    test_expert_combined = np.append(X_test, expert_test, axis=1)
    
    
    expert_train=expert_train.reshape(expert_train.shape[0],horizon)
    expert_test=expert_test.reshape(expert_test.shape[0],horizon)
    
    ops.reset_default_graph()
    K.clear_session()

    inputs_train= Input(batch_shape=(None,window_sz+horizon, 1))
    inputs_expert = Input(batch_shape=(None, horizon))
    layer1 = LSTM(4, activation='relu', kernel_initializer=glorot_uniform(3))(inputs_train)
    layer2 = Dense(horizon, activation='linear')(layer1)
    residual = Add()([layer2, inputs_expert])

    model = tf.keras.Model(inputs=[inputs_train, inputs_expert], outputs=residual, name ="covid")
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), 
                  loss='mse', metrics=['mse'])
    
    model.fit([train_expert_combined, expert_train], y_train, shuffle=False, epochs=50, batch_size=20, verbose=0)

    
    preds = model.predict(([test_expert_combined, expert_test]))

    preds=preds.flatten()[:50]
    error = mape(test_ts[-50:],preds)
    mape_ts.append(error)



In [ ]:
np.mean(mape_ts)